In [60]:
import pandas as pd
import numpy as np
import sklearn.metrics as mt
from gensim.models import Word2Vec
from scipy import spatial

model = Word2Vec.load('/Users/dakotamurray/Dropbox/SME-dropbox/Archive/Embeddings_20191209/word2vec/2008-2019_word2vec_d200_ws2_model.bin')
meta = pd.read_csv('/Users/dakotamurray/Dropbox/SME-dropbox/Data/Additional/institution_lookup_with_states.txt', sep = "\t")

meta = meta.fillna(-1)
meta = meta.astype({'cwts_org_no': int})
meta = meta.astype({'cwts_org_no': str})

# Limit the metadata to orgs that are actually in the w2v model vocabulary
vocab = list(model.wv.vocab.keys())
#meta = meta.loc[meta.cwts_org_no in vocab]
meta = meta.loc[meta.cwts_org_no.isin(vocab)]

,cwts_org_no,wos_name,full_name,city,country_iso_num_code,latitude,longitude,org_type_code,org_type,country_iso_name,country_iso_alpha,city_country,region
1,1011,CANAKKALE ONSEKIZ MART UNIV,Canakkale Onsekiz Mart University,CANAKKALE,792.0,40.113700,26.408700,U,University,Turkey,TUR,CANAKKALE_TUR,Çanakkale
2,10472,HOP SUD GRENOBLE,"Hopital Sud, Grenoble",eCHIROLLES,250.0,45.148107,5.729546,H,Hospital,France,FRA,eCHIROLLES_FRA,Auvergne-Rhône-Alpes
4,10026,HITITE UNIV,Hittite University,CORUM,792.0,40.570902,34.978872,U,University,Turkey,TUR,CORUM_TUR,Çorum
9,31891,AALESUND UNIV COLL,Aalesund University College,ALESUND,578.0,62.472173,6.235352,E,Teaching Organisation,Norway,NOR,ALESUND_NOR,Møre og Romsdal
10,31911,INLAND NORWAY UNIV APPLI SCI,Inland Norway University of Applied Sciences,AS,578.0,60.369136,5.349631,E,Teaching Organisation,Norway,NOR,AS_NOR,Hordaland
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10412,15577,DISNEY RES ZURICH,Disney Research Zurich,ZURICH,756.0,47.380548,8.542793,R,Research organisation,Switzerland,CHE,ZURICH_CHE,Zurich
10413,15583,FED OFF METEOROL & CLIMATOL METEOSWISS,Federal Office of Meteorology and Climatology ...,ZURICH,756.0,47.454106,8.564473,R,Research organisation,Switzerland,CHE,ZURICH_CHE,Zurich
10418,984,TECH UNIV SVOLEN,"Technical University, Zvolen",ZVOLEN,703.0,48.708456,19.137943,U,University,Slovakia,SVK,ZVOLEN_SVK,Region of Banská Bystrica
10420,2846,WINDESHEIM UNIV APPL SCI,Windesheim University of Applied Sciences,ZWOLLE,528.0,52.498588,6.079925,E,Teaching Organisation,Netherlands,NLD,ZWOLLE_NLD,Overijssel


In [81]:
# first, lets do Canada
# limit meta to orgs actually in our data
canada_meta = meta.loc[meta['country_iso_alpha'] == 'CAN']
canada_meta

quebec_meta = canada_meta.loc[canada_meta['region'] == 'Quebec']
rest_canada_meta = canada_meta.loc[canada_meta['region'] != 'Quebec']
france_meta = meta.loc[meta['country_iso_alpha'] == 'FRA']

quebec_vec = np.array([model.wv[x] for x in (quebec_meta.cwts_org_no)]).mean(axis = 0)
canada_vec = np.array([model.wv[x] for x in (canada_meta.cwts_org_no)]).mean(axis = 0)
france_vec = np.array([model.wv[x] for x in (france_meta.cwts_org_no)]).mean(axis = 0)


print('Distnace between Quebec and Rest of Canada:', 1 - spatial.distance.cosine(quebec_vec, canada_vec))
print('Distnae between Quebec and France:', 1 - spatial.distance.cosine(quebec_vec, france_vec))

Distnace between Quebec and Rest of Canada: 0.88188636302948
Distnae between Quebec and France: 0.650307834148407


In [109]:
mass = meta.loc[meta['region'] == 'Massachusetts']

#umass = mass.loc['University of Mass'.isin(mass['full_name'])]

# First, get all pairwise distances and save to a file
umass = mass.loc[mass['full_name'].str.contains('University of Mass')]
umass = umass.loc[mass.cwts_org_no != '1226']
boston = mass.loc[mass.city == 'Boston']
boston = mass.loc[mass.cwts_org_no != '1226']


umass_vec = np.array([model.wv[x] for x in (umass.cwts_org_no)]).mean(axis = 0)
boston_vec = np.array([model.wv[x] for x in (boston.cwts_org_no)]).mean(axis = 0)
umassboston_vec = model.wv['1226']

print('Distnace between UMass Boston and Rest UMass System:', 1 - spatial.distance.cosine(umassboston_vec, umass_vec))
print('Distnae between UMAss Boston and rest of Boston:', 1 - spatial.distance.cosine(umassboston_vec, boston_vec))

Distnace between UMass Boston and Rest UMass System: 0.3462038040161133
Distnae between UMAss Boston and rest of Boston: 0.4126112461090088
